In [1]:
import numpy as np
import cv2
from numba import njit, vectorize, float64, int64, prange

In [24]:
def constructIndices(img):
    args = np.argwhere(img > 0)
    m, n = img.shape
    I = args[:, 1]
    J = args[:, 0]
    return I, J

In [25]:
@vectorize([int64(int64, int64, int64, int64)], nopython=True)
def customMulti(I, J, theta, rho_max):
    
    theta = np.radians(theta)
    
    rho = J * np.cos(theta) + I * np.sin(theta) + rho_max

    return rho

In [26]:
@njit
def fillSpace(rho, theta, space):
    for r in rho:
        space[r, theta] += 1

In [5]:
@njit
def buildSpace(I, J, rho_max):   
    
    space = np.zeros((2 * rho_max, 180))
    
    for theta in prange(180):
        rho = customMulti(I, J, theta, rho_max)
        fillSpace(rho,theta, space)
    
    return space
        

In [329]:
def findMaxima(hSpace, numOfMaxima, dr, dt):
    maxs = []
    
    for i in range(numOfMaxima):
        
        mx = np.argwhere(hSpace == np.max(hSpace))[0]
        maxs.append(mx)
        r, t = mx[0], mx[1]
        hSpace[r, t] = 0
        
        
    return maxs


def drawRandomLines(NbLines):
    img = np.zeros((800, 800))
    for i in range(NbLines):
        x1 = np.random.randint(low = 0, high=800)
        y1 = np.random.randint(low = 0, high=800)
        x2 = np.random.randint(low = 0, high=800)
        y2 = np.random.randint(low = 0, high=800)
        cv2.line(img, (x1, y1), (x2, y2), (255,255,255), 1)
    img = np.float32(img)
    
    return img

def drawLine(imgC, mx, rho_max, m, n):

    theta = np.radians(mx[1])
    rho = mx[0] - rho_max

    c = np.cos(theta)
    s = np.sin(theta)
    
    if np.isclose(s, 0):
        pt1 = (0, rho)
        pt2 = (n, rho)

    else:
        pt1 = (int(rho // s), 0)
        pt2 = (int(rho // s - c / s * m), m)

    cv2.line(imgC, pt1, pt2, (255,0,255), 3)


def show(I, title):
    I = np.uint8(I / np.max(I) * 255)
    cv2.imshow(title, I)
    cv2.waitKey()
    cv2.destroyAllWindows()

In [341]:
#img = drawRandomLines(NbLines=5)

img = cv2.imread('tst5.png', 0)
imgC = cv2.imread('tst5.png')
edges = cv2.Canny(img, 80, 200)

show(imgC, 'Original')

I, J = constructIndices(edges)
m, n = img.shape
rho_max = int(np.hypot(m, n)) 

hs = buildSpace(I, J, rho_max)

mxs = findMaxima(hs, 15, 1, 1)

for mx in mxs:
    
    drawLine(imgC, mx = mx, rho_max = rho_max, m = m, n = n)

show(imgC, 'Detected')